# Model AI: Basic, Ideal, Plug-Flow
Eq. 21

$$ u \cdot \rho_f \cdot c_{pf}\frac{dT}{dZ}-\rho_b\sum_i\left(-\Delta H_i\right) \cdot r_i\left(C, T\right) = 0 $$

$$T(Z = 0) = T_0$$

In [1]:
import cantera as ct
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [2]:
gas = ct.Solution("input/han.cti")
surf = ct.Interface('input/han.cti','surface1', [gas])

In [3]:
temp = 20 + 273.15
pres = ct.one_atm
comp = 'H4N2O2(2):0.14, NH2OH(3):0.3, HNO3(4):0.3, CH3OH(5):0.16, H2O(6):0.04'

gas.TPX = temp, pres, comp
surf.TP = temp, pres

In [4]:
# Parameters
u = 1                       # flow velocity (m/s)
pf = 1000 / gas.volume_mole # fluid molar concentration (mol/m^3)
cpf = gas.cp_mole / 1000    # fluid specific heat capacity (J/mol-K)
epsilon = .9                # void volume of reactor
pc = 4000                   # density of catalyst (kg/m^3)
pb = (1 - epsilon) * pc     # solid density

In [5]:
def gas_rate(z, CT):
    C, T = CT[:-1], CT[-1]
    gas.X = C
    dC = pb/u * gas.net_production_rates
    dT = pb/(u*pf*cpf) * np.dot(-gas.delta_enthalpy, gas.net_rates_of_progress)
    
    return np.append(dC, dT)

In [6]:
gas_C0 = gas.X
T0 = 600
L = 10 # reactor length
gas_initial_values = np.append(gas_C0, T0)

In [7]:
gas_solution = solve_ivp(gas_rate, (0, L), gas_initial_values)

KeyboardInterrupt: 

### Concentration Profile of OH

In [ ]:
plt.plot(gas_solution.t, gas_solution.y[gas.species_index("OH(19)")])

### Temperature Profile

In [ ]:
plt.plot(gas_solution.t, gas_solution.y[-1])

In [ ]:
def rate_AI(z, CT):
    gas_C, surf_C, T = CT[:gas.X.shape[0]], CT[gas.X.shape[0]:-1], CT[-1] 
    gas.X = gas_C
    surf.X = surf_C
    enthalpies = np.append(gas.delta_enthalpy, surf.delta_enthalpy)
    rates = np.append(gas.net_rates_of_progress, surf.net_rates_of_progress)
    dC = pb/u * surf.net_production_rates
    dT = pb/(u*pf*cpf) * np.dot(-enthalpies, rates)
    
    return np.append(dC, dT)

In [ ]:
surf_C0 = surf.X
initial_values = np.concatenate([gas_C0, surf_C0, [T0]])

In [ ]:
solution = solve_ivp(rate_AI, (0, L), initial_values)

In [ ]:
plt.plot(gas_solution.t, gas_solution.y[gas.species_index("OH(19)")])

In [ ]:
plt.plot(gas_solution.t, gas_solution.y[-1])